In [1]:
from matplotlib import pyplot
from math import pi
import numpy
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
import numba
from numba import jit

In [2]:
def uex(t,X,Y):
    return -numpy.exp(-2*t)*numpy.cos(X)*numpy.sin(Y)

def vex(t,X,Y):
    return numpy.exp(-2*t)*numpy.sin(X)*numpy.cos(Y)

def pex(t,X,Y):
    return -numpy.exp(-4*t)/4.*(numpy.cos(2*X)+numpy.cos(2*Y))

In [3]:
def Fc1(u,v,dx,dy):
    F=numpy.zeros(numpy.shape(u))
    F[1:-1,1:-1]=-((u[1:-1,1:-1]+u[1:-1,2:])**2-(u[1:-1,1:-1]+u[1:-1,:-2])**2)/(4*dx)-\
                ((u[1:-1,1:-1]+u[2:,1:-1])*(v[1:-1,2:]+v[1:-1,1:-1])-(u[1:-1,1:-1]+u[:-2,1:-1])*(v[:-2,2:]+v[:-2,1:-1]))/(4*dy)
        
    F[:,0]=F[:,-2]
    F[:,-1]=F[:,1]
    F[0,:]=F[-2,:]
    F[-1,:]=F[-1,:]
    return F

In [4]:
def Fv1(u,v,dx,dy):
    F=numpy.zeros(numpy.shape(u))
    F[1:-1,1:-1]=(u[1:-1,2:]-2*u[1:-1,1:-1]+u[1:-1,:-2])/dx**2+(u[2:,1:-1]-2*u[1:-1,1:-1]+u[:-2,1:-1])/dy**2
    
    F[:,0]=F[:,-2]
    F[:,-1]=F[:,1]
    F[0,:]=F[-2,:]
    F[-1,:]=F[-1,:]
    return F

In [5]:
def Fp1(p,dx,dy):
    F=numpy.zeros(numpy.shape(p))
    F[1:-1,1:-1]=-(p[1:-1,2:]-p[1:-1,1:-1])/dx
    F[:,0]=F[:,-2]
    F[:,-1]=F[:,1]
    F[0,:]=F[-2,:]
    F[-1,:]=F[-1,:]
    return F

In [6]:
def Fc2(u,v,dx,dy):
    F=numpy.zeros((numpy.shape(u)))
    F[1:-1,1:-1]=-((u[2:,1:-1]+u[1:-1,1:-1])*(v[1:-1,1:-1]+v[1:-1,2:])-(u[2:,:-2]+u[1:-1,:-2])*(v[1:-1,1:-1]+v[1:-1,:-2]))/(4*dx)-\
                 ((v[1:-1,1:-1]+[v[2:,1:-1]])**2-(v[1:-1,1:-1]+v[:-2,1:-1])**2)/(4*dy)
        
    F[:,0]=F[:,-2]
    F[:,-1]=F[:,1]
    F[0,:]=F[-2,:]
    F[-1,:]=F[-1,:]
    return F

In [7]:
def Fv2(u,v,dx,dy):
    F=numpy.zeros((numpy.shape(u)))
    F[1:-1,1:-1]=(v[1:-1,2:]-2*v[1:-1,1:-1]+v[1:-1,:-2])/dx**2+(v[2:,1:-1]-2*v[1:-1,1:-1]+v[:-2,1:-1])/dy**2
        
    F[:,0]=F[:,-2]
    F[:,-1]=F[:,1]
    F[0,:]=F[-2,:]
    F[-1,:]=F[-1,:]
    return F

In [8]:
def Fp2(p,dx,dy):
    F=numpy.zeros((numpy.shape(p)))
    F[1:-1,1:-1]=-(p[2:,1:-1]-p[1:-1,1:-1])/dy
        
    F[:,0]=F[:,-2]
    F[:,-1]=F[:,1]
    F[0,:]=F[-2,:]
    F[-1,:]=F[-1,:]
    return F

In [9]:
@jit
def poisson(f,res_target,omega):
    nj,ni=numpy.shape(f)-numpy.array((1,1))
    dx=2*pi/(ni-1)
    dy=2*pi/(nj-1)
    x=numpy.linspace(-dx/2,2*pi+dx/2,ni+1)
    y=numpy.linspace(-dy/2,2*pi+dy/2,nj+1)
    X,Y=numpy.meshgrid(x,y)
    
    p=numpy.zeros(numpy.shape(f))
    res=numpy.zeros(numpy.shape(f))
    
    res_max=1e3
    n=0
    while res_max>res_target:
        
        for j in range(1,nj):
            for i in range(1,ni):
                p[j,i]=omega/(2*(dx**2+dy**2))*((p[j,i-1]+p[j,i+1])*dy**2+\
                       (p[j-1,i]+p[j+1,i])*dx**2-f[j,i]*dx**2*dy**2)+\
                       (1-omega)*p[j,i]
        p[0,:]=p[1,:]
        p[-1,:]=p[-2,:]
        p[:,0]=p[:,1]
        p[:,-1]=p[:,-2]
        n+=1
        
        for j in range(1,nj):
            for i in range(1,ni):
                res[j,i]=numpy.abs(f[j,i]-(p[j,i-1]-2*p[j,i]+p[j,i+1])/dx**2-\
                                  (p[j-1,i]-2*p[j,i]+p[j+1,i])/dy**2)
        res_max=numpy.max(res)
    
    return p

In [10]:
@jit
def getuv(w1,w2,p,dx,dy,dt):
    u=w1+Fp1(p,dx,dy)*dt
    v=w2+Fp2(p,dx,dy)*dt
    
    u[0,:]=-u[1,:]
    u[-1,:]=-u[-2,:]
    u[:,0]=u[:,-2]
    u[:,-1]=u[:,1]

    v[0,:]=v[-2,:]
    v[-1,:]=v[1,:]
    v[:,0]=-v[:,1]
    v[:,-1]=-v[:,-2]
    return u,v

In [11]:
@jit
def error(p,p_ex):
    nj,ni=numpy.shape(p)-numpy.array((1,1))
    err=numpy.zeros(numpy.shape(p))
    for j in range(nj+1):
        for i in range(ni+1):
            err[j,i]=numpy.abs(p[j,i]-p_ex[j,i])
    err_max=numpy.mean(err[2:-2,2:-2])
    return err_max

In [12]:
@jit
def RK3(ni,nj,dt):
    T=2.
    t=0.
    dx=2.*pi/(ni-1)
    dy=2.*pi/(nj-1)
    x=numpy.linspace(-dx/2,2*pi+dx/2,ni+1)
    y=numpy.linspace(-dy/2,2*pi+dy/2,nj+1)
    xu=numpy.linspace(0,2*pi+dx,ni+1)
    yu=numpy.linspace(-dy/2,2*pi+dy/2,nj+1)
    xv=numpy.linspace(-dx/2,2*pi+dx/2,ni+1)
    yv=numpy.linspace(0,2*pi+dy,nj+1)

    X,Y=numpy.meshgrid(x,y)
    Xu,Yu=numpy.meshgrid(xu,yu)
    Xv,Yv=numpy.meshgrid(xv,yv)

    u0=uex(0,Xu,Yu)
    v0=vex(0,Xv,Yv)
    p0=pex(0,X,Y)
            
    u=u0.copy()
    v=v0.copy()
    p=p0.copy()
    res=numpy.zeros((nj+1,ni+1))
    nt=0
    
    while t<T:
        G1=numpy.zeros((nj+1,ni+1))
        G2=numpy.zeros((nj+1,ni+1))
        w1=numpy.zeros((nj+1,ni+1))
        w2=numpy.zeros((nj+1,ni+1)) 
        f=numpy.zeros((nj+1,ni+1))
        
        #from t to t+dt/3
        #t+=dt/3
        G1=Fc1(u,v,dx,dy)+Fv1(u,v,dx,dy)
        G2=Fc2(u,v,dx,dy)+Fv2(u,v,dx,dy)
        w1=u+dt/3*G1
        w2=v+dt/3*G2
        f[1:-1,1:-1]=3./dt*((w1[1:-1,1:-1]-w1[1:-1,:-2])/dx+(w2[1:-1,1:-1]-w2[:-2,1:-1])/dy)
        p=poisson(f,1e-6,1.99)
        u,v=getuv(w1,w2,p,dx,dy,dt/3)
        
        #from t+dt/3 to t+3dt/4
        #t+=5/12*dt
        G1=-5./9*G1+Fc1(u,v,dx,dy)+Fv1(u,v,dx,dy)
        G2=-5./9*G2+Fc2(u,v,dx,dy)+Fv2(u,v,dx,dy)
        w1=u+dt*15./16*G1
        w2=v+dt*15./16*G2
        f[1:-1,1:-1]=12./5/dt*((w1[1:-1,1:-1]-w1[1:-1,:-2])/dx+(w2[1:-1,1:-1]-w2[:-2,1:-1])/dy)        
        p=poisson(f,1e-6,1.99)
        u,v=getuv(w1,w2,p,dx,dy,5.*dt/12)
        
        #from t+3dt/4 to t+dt
        #t+=dt/4
        G1=-153./128*G1+Fc1(u,v,dx,dy)+Fv1(u,v,dx,dy)
        G2=-153./128*G2+Fc2(u,v,dx,dy)+Fv2(u,v,dx,dy)
        w1=u+dt*8./15*G1
        w2=v+dt*8./15*G2
        f[1:-1,1:-1]=4./dt*((w1[1:-1,1:-1]-w1[1:-1,:-2])/dx+(w2[1:-1,1:-1]-w2[:-2,1:-1])/dy)
        p=poisson(f,1e-6,1.99)
        u,v=getuv(w1,w2,p,dx,dy,dt/4)
        
        
        if (int(t*10000))%1000==0:
            print(int(t/dt),error(u,uex(t,Xu,Yu)),error(v,vex(t,Xv,Yv)),error(p,pex(t,X,Y)))
        nt+=1
        t+=dt
        
    A=numpy.array([u,v,p])
    return A

In [13]:
def grid_p(ni,nj):
    dx=2*pi/(ni-1)
    dy=2*pi/(nj-1)
    x=numpy.linspace(-dx/2,2*pi+dx/2,ni+1)
    y=numpy.linspace(-dy/2,2*pi+dy/2,nj+1)
    X,Y=numpy.meshgrid(x,y)
    return X,Y

def grid_u(ni,nj):
    dx=2*pi/(ni-1)
    dy=2*pi/(nj-1)
    xu=numpy.linspace(0,2*pi+dx,ni+1)
    yu=numpy.linspace(-dy/2,2*pi+dy/2,nj+1)
    X,Y=numpy.meshgrid(xu,yu)
    return X,Y

def grid_v(ni,nj):
    dx=2*pi/(ni-1)
    dy=2*pi/(nj-1)
    xv=numpy.linspace(-dx/2,2*pi+dx/2,ni+1)
    yv=numpy.linspace(0,2*pi+dy,nj+1)
    X,Y=numpy.meshgrid(xv,yv)
    return X,Y

In [14]:
X,Y=grid_p(41,41)
Xu,Yu=grid_u(41,41)
Xv,Yv=grid_v(41,41)

In [15]:
A1_1=RK3(41,41,4e-4)
u_err=error(A1_1[0],uex(2,Xu,Yu))
v_err=error(A1_1[1],vex(2,Xv,Yv))
p_err=error(A1_1[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

0 0.000327971193168 0.000327971193166 0.203895299316
500 6.45586352476e-06 6.45586352558e-06 0.0917859768869
750 9.79827382585e-05 9.79827382599e-05 0.0615829986216
1000 0.000156146658284 0.000156146658283 0.0413185704494
1250 0.000190029896068 0.000190029896064 0.0277223281944
u error=5.605626254361603e-05
v error=5.605626271830215e-05
p error=6.967921329384435e-05


In [16]:
A1_2=RK3(41,41,2e-4)
u_err=error(A1_2[0],uex(2,Xu,Yu))
v_err=error(A1_2[1],vex(2,Xv,Yv))
p_err=error(A1_2[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

0 0.000164018326319 0.000164018326318 0.204032865582
500 3.94258291704e-06 3.94258291629e-06 0.136894083975
1000 0.000116447111441 0.000116447111442 0.0918478910727
u error=5.908394342001041e-05
v error=5.9083943506728404e-05
p error=6.972625659109767e-05


In [17]:
A1_3=RK3(41,41,1e-4)
u_err=error(A1_3[0],uex(2,Xu,Yu))
v_err=error(A1_3[1],vex(2,Xv,Yv))
p_err=error(A1_3[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

0 8.2017347227e-05 8.20173472264e-05 0.204101710572
1000 7.11068982442e-05 7.11068982439e-05 0.136940270093
2000 0.000171538155633 0.000171538155633 0.0918422168783
3000 0.000233177698548 0.000233177698549 0.0616207258239
4000 0.000266865244428 0.000266865244428 0.0413438788229
5000 0.000280703437099 0.000280703437098 0.0277393056922
6000 0.000280776883394 0.000280776883394 0.0186114372429
7000 0.000271617343395 0.000271617343394 0.0124871749391
8000 0.000256566614438 0.00025656661444 0.00837815591056
9000 0.000238058907457 0.000238058907456 0.0056212466045
10000 0.000217839973835 0.000217839973834 0.00377152331474
11000 0.00019713663066 0.00019713663066 0.00253046837231
12000 0.000176787463462 0.000176787463462 0.00169779397969
13000 0.000157343207848 0.000157343207845 0.00113911880929
14000 0.000139143505754 0.000139143505753 0.000764280925415
15000 0.000122375301287 0.000122375301266 0.000512787041097
16000 0.000107117008649 0.00010711700866 0.000344049513233
17000 9.33716907329e-05

In [18]:
X,Y=grid_p(81,81)
Xu,Yu=grid_u(81,81)
Xv,Yv=grid_v(81,81)

In [19]:
A2_1=RK3(81,81,4e-4)
u_err=error(A2_1[0],uex(2,Xu,Yu))
v_err=error(A2_1[1],vex(2,Xv,Yv))
p_err=error(A2_1[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

0 0.000326800083967 0.000326800083967 0.177244116611
500 0.000162756831393 0.000162756831388 0.0796760280034
750 0.000110201636404 0.000110201636385 0.0534202108057
1000 7.13498307773e-05 7.13498307162e-05 0.0358165308945
1250 4.29606468734e-05 4.29606467713e-05 0.0240138305961
u error=9.412989543689925e-06
v error=9.412995787303639e-06
p error=5.972293066109929e-05


In [20]:
A2_2=RK3(81,81,2e-4)
u_err=error(A2_2[0],uex(2,Xu,Yu))
v_err=error(A2_2[1],vex(2,Xv,Yv))
p_err=error(A2_2[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

0 0.000163432705193 0.000163432705193 0.177358004581
500 9.94105807708e-05 9.94105807705e-05 0.11891282978
1000 5.32258860339e-05 5.32258860314e-05 0.0797272222136
u error=1.241132428947986e-05
v error=1.241132741104864e-05
p error=5.9761521850812196e-05


In [21]:
A2_3=RK3(81,81,1e-4)
u_err=error(A2_3[0],uex(2,Xu,Yu))
v_err=error(A2_3[1],vex(2,Xv,Yv))
p_err=error(A2_3[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

0 8.17245200317e-05 8.17245200318e-05 0.177414999849
1000 3.25067001322e-05 3.25067001321e-05 0.118951042707
2000 1.58386415357e-06 1.5838641547e-06 0.0797209651812
3000 2.43583640334e-05 2.43583640375e-05 0.0534503389885
4000 3.882612842e-05 3.88261284342e-05 0.0358367303818
5000 4.72499802319e-05 4.72499802552e-05 0.0240273734262
6000 5.13453928839e-05 5.13453929377e-05 0.0161095803008
7000 5.24045609822e-05 5.24045611408e-05 0.0108009557712
8000 5.13934776774e-05 5.13934780145e-05 0.00724169334668
9000 4.9027728056e-05 4.90277284702e-05 0.00485532305013
10000 4.58315239796e-05 4.58315246653e-05 0.00325533807961
11000 4.21835663776e-05 4.21835671506e-05 0.00218259999352
12000 3.83525690807e-05 3.83525699264e-05 0.00146336351975
13000 3.45246864961e-05 3.45246874376e-05 0.000981138792002
14000 3.08246122406e-05 3.08246132754e-05 0.000657822680857
15000 2.73317418486e-05 2.73317430133e-05 0.000441049493912
16000 2.40924960028e-05 2.40924972408e-05 0.00029571003077
17000 2.11296647299e-

In [ ]:
X,Y=grid_p(121,121)
Xu,Yu=grid_u(121,121)
Xv,Yv=grid_v(121,121)

In [ ]:
A3_1=RK3(121,121,4e-4)
u_err=error(A3_1[0],uex(2,Xu,Yu))
v_err=error(A3_1[1],vex(2,Xv,Yv))
p_err=error(A3_1[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

In [ ]:
A3_2=RK3(121,121,2e-4)
u_err=error(A3_2[0],uex(2,Xu,Yu))
v_err=error(A3_2[1],vex(2,Xv,Yv))
p_err=error(A3_2[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

In [ ]:
A3_2=RK3(121,121,1e-4)
u_err=error(A3_2[0],uex(2,Xu,Yu))
v_err=error(A3_2[1],vex(2,Xv,Yv))
p_err=error(A3_2[2],pex(2,X,Y))
    
print('u error={}'.format(u_err))
print('v error={}'.format(v_err))
print('p error={}'.format(p_err))

In [ ]:
pyplot.figure(figsize=(14,5))
pyplot.subplot(121)
pyplot.contour(Xu,Yu,A3_3[0],10)
pyplot.colorbar()
pyplot.xlabel('$x$')
pyplot.ylabel('$y$')
pyplot.title('$u$ velocity')

pyplot.subplot(122)
pyplot.contour(Xv,Yv,A3_3[1],10)
pyplot.colorbar()
pyplot.xlabel('$x$')
pyplot.ylabel('$y$')
pyplot.title('$v$ velocity');

In [ ]:
pyplot.figure(figsize=(6.5,5))
pyplot.contour(X,Y,A3_3[2],10)
pyplot.colorbar()
pyplot.xlabel('$x$')
pyplot.ylabel('$y$')
pyplot.title('$p$ pressure');